In [2]:
import sys
if 'google.colab' in sys.modules:
    !pip install langchain google-cloud-aiplatform rich
    !pip install wikipedia

In [151]:
import langchain
langchain.debug = False

# Imports

In [148]:
import os
import requests
from google.cloud import aiplatform
from langchain.agents import (AgentExecutor, AgentType, BaseSingleActionAgent,
                              Tool, initialize_agent)
from langchain.llms import VertexAI
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool
from pydantic import BaseModel, Field
from rich import print
from rich.console import Console
from rich.progress import Progress, track
from rich.table import Table


NameError: name 'langchain' is not defined

# Google Authentication

In [4]:
tool_url = None #!TODO: set this to the tool url if using Colab
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    import dotenv
    dotenv.load_dotenv()
    tool_url = str(os.getenv('TOOL_URL'))
    pass

In [5]:
#! Change the following to your own project and location
aiplatform.init(project="aerobic-gantry-387923", location="us-central1")
llm = VertexAI()


# Prompt Template

In [164]:
from langchain.agents import (
    Tool,
    AgentExecutor,
    LLMSingleActionAgent,
    AgentOutputParser,
)
from langchain.prompts import StringPromptTemplate
# Set up the base template
template = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}], 
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a king , compasstionate Guidance Counselor when giving your final answer. Use lots of "Arg"s



Question: {input}
{agent_scratchpad}"""

## Custome Prompt Tool Retreiver

In [131]:
from typing import Callable


# Set up a prompt template
class CustomPromptTemplateToolRetreiver(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

## Custome Prompt Tool

In [132]:
from typing import List, Union
# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

# Output Parser

In [133]:
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
import re
from langchain.schema import AgentAction, AgentFinish, OutputParserException
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)
output_parser = CustomOutputParser()

# Vector Stores
1. Websites  using The Webase Loader

In [134]:
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
'''
urls = [
    "https://www.utdallas.edu/galaxy/",
    "https://www.utdallas.edu/costs-scholarships-aid/"
    "https://www.utdallas.edu/academics/"
]
loader = WebBaseLoader(urls)


# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()
docs = loader.aload()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")
'''

'\nurls = [\n    "https://www.utdallas.edu/galaxy/",\n    "https://www.utdallas.edu/costs-scholarships-aid/"\n    "https://www.utdallas.edu/academics/"\n]\nloader = WebBaseLoader(urls)\n\n\n# fixes a bug with asyncio and jupyter\nimport nest_asyncio\n\nnest_asyncio.apply()\ndocs = loader.aload()\n\ntext_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)\ntexts = text_splitter.split_documents(docs)\n\nembeddings = OpenAIEmbeddings()\ndocsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")\n'

In [11]:
'''
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)
'''

'\nstate_of_union = RetrievalQA.from_chain_type(\n    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()\n)\n'

# Tools Function

In [135]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

def request_error_handler(url: str, params={}) -> str:
    try:
        response = requests.get(url, params)
        response.raise_for_status()  # Raise an HTTPError if the status code is not in the 200 range
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        return e

## Get a Tool Function

In [136]:
def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [ALL_TOOLS[d.metadata["index"]] for d in docs]

## Courses

In [137]:
def get_courses_from_query(query: str) -> str:
    url = f"https://{tool_url}/get-possible-courses/{query}"
    return request_error_handler(url)


## Degree

In [102]:
def get_degrees_from_query(query: str) -> str:
    url = f"https://{tool_url}/get-possible-degrees/{query}"
    return request_error_handler(url)


## Utdallas Search

In [138]:
def utdallas_search(query: str) -> str:
    url = f"https://{tool_url}/search/{query}"
    return request_error_handler(url)

def access_utdallas_site(url_to_search: str) -> str:
    url = f"https://{tool_url}/access/"
    params = {
        "url": url_to_search
    }
    return request_error_handler(url, params)



## Wikipedia

In [104]:
from langchain.utilities import WikipediaAPIWrapper
wikipedia = WikipediaAPIWrapper()
def run_wikepedia(query: str) -> str:
    return wikipedia.run(query)

## Dictionary


In [139]:
def dictionary_search(query: str) -> str:
    url = f"https://{tool_url}/dictionary/{query}"
    return request_error_handler(url)

# Tools NAME and description

In [140]:
tools = [
    Tool(
        name = "UTD Degree Search",
        func = get_degrees_from_query,
        description = "This tool allows you to search for UT Dallas' Degrees, Majors, Minors, and Certificates using keywords. Use this tool to compare multiple degrees. It returns relevant URLs, titles, and snippets from the search engine results.",
    ),
    Tool(
        name = "UTD Course Search",
        func = get_courses_from_query,
        description = "This tool enables you to search for UT Dallas' courses using keywords. Use this tool to compare multiple courses. It returns the course numbers/codes where the keyword exists in the course title.",
    ),

    Tool(
        name = "UTD search",
        func = utdallas_search,
        description = "Please use this tool only as a last resort. It performs a comprehensive search across all UT Dallas pages that will not give information about courses or degrees. It requires a query string and returns possible links and snippet contents. Do not use for course search or degree search as there are separate tools for those.",
    ),
    Tool(
        name = "Extract Text from utdallas.edu site from Course, Degree, or Website Search Tool",
        func = access_utdallas_site,
        description = "If you have a URL string containing utdallas.edu, use this function to extract the text content from that page.",
    ),
    Tool(
        name = "Dictionary Search",
        func = dictionary_search,
        description = "If you need the definition of a word or phrase unrelated to UT Dallas, use this function to retrieve the definition. Do not use for anything related to UT Dallas' courses.",
    )
]






## Set up Tools for Retreiver

In [171]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

# Initialize Agents

## Zero Shot ReAcT Agents


In [172]:
'''
agent = initialize_agent(
    tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory = memory ,
    max_execution_time = 10 , return_intermediate_steps=True,
)
'''
memory = ConversationBufferMemory(memory_key="chat_history")
zero_shot_react_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
    max_execution_time = 20 ,
    memory = memory,
)

## Plan and execute

In [145]:
from langchain.experimental import plan_and_execute
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools = tools , verbose=True)
plan_and_execute = PlanAndExecute (
    planner=planner,
    executor=executor,
    verbose=True,
    max_execution_time = 20 ,
    return_intermediate_steps=True,
)




## Custome Agents

In [169]:
# LLM chain consisting of the LLM and a prompt
from langchain import  LLMChain

llm_chain = LLMChain(llm=llm, 
                     prompt=prompt, 
                     memory=ConversationBufferMemory(memory_key="chat_history"), 
                     verbose=True)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names , 
    max_execution_time = 20 ,
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Testing Agents

## Tell me the Difference between a courses

In [173]:
response = zero_shot_react_agent(
        {
            "input": "What is the difference between Math 2417 and Math 2419"
        }
)
print(
    response["output"]
)






> Entering new  chain...
I need to know more about Math 2417 and Math 2419
Action: UTD Course Search
Action Input: Math 2417
Observation: {'course_ids': {'items': [{'title': 'Calculus I - UT Dallas 2022 Undergraduate Catalog - The University ...', 'link': 'https://catalog.utdallas.edu/2022/undergraduate/courses/math2417', 'snippet': 'MATH2417 - Calculus I. MATH 2417 Calculus I (4 semester credit hours) Functions, limits, continuity, differentiation; integration of function of one\xa0...'}, {'title': 'Calculus I - UT Dallas 2022 Graduate Catalog - The University of ...', 'link': 'https://catalog.utdallas.edu/2022/graduate/courses/math2417', 'snippet': 'MATH2417 - Calculus I. MATH 2417 Calculus I (4 semester credit hours) Functions, limits, continuity, differentiation; integration of function of one\xa0...'}, {'title': 'Calculus II - UT Dallas 2017 Undergraduate Catalog - The ...', 'link': 'https://catalog.utdallas.edu/2017/undergraduate/courses/math2419', 'snippet': 'MATH 2419 Calculu

ValueError: One output key expected, got dict_keys(['output', 'intermediate_steps'])

In [170]:
agent_executor = agent_executor.run("What is the difference between Math 2417 and Math 2419")
print(
    agent_executor["output"]
)



> Entering new  chain...


> Entering new  chain...


KeyError: 'intermediate_steps'

In [162]:
try:
  plan_execute_response = plan_and_execute(
    "What is the difference between Math 2417 and Math 2419"
  )
  print(
    plan_execute_response["output"]
  )
except Exception as e:
  print(e)







> Entering new  chain...
steps=[Step(value='Go to the course catalog'), Step(value='Search for Math 2417'), Step(value='Read the course description'), Step(value='Search for Math 2419'), Step(value='Read the course description'), Step(value='Compare the two course descriptions'), Step(value='Given the above steps taken, please respond to the users original question\n')]

> Entering new  chain...


Could not parse LLM output: Question: What is the course catalog?
Action:
```
{
  "action": UTD search,
  "action_input": "course catalog"
}
```

## Contacting People at my University

In [44]:
zero_shot_react_agent_response = zero_shot_react_agent(
    {
        "input": "What is the phone number to contact for Accounting advising?"
    }
)
print(
    zero_shot_react_agent_response["output"]
)




> Entering new  chain...
I don't know what you are talking about.
Action: UTD Search
Action Input: accounting advising
Observation: {'items': [{'title': 'Academic Advising | Naveen Jindal School of Management - Dallas', 'link': 'https://jindal.utdallas.edu/advising/', 'snippet': 'JSOM Academic Advising is here to help guide you by providing you with new student guidelines, registration steps, and graduation information. Read more.'}, {'title': 'MS Accounting Registration/CPA Advising Session - Comet Calendar', 'link': 'https://calendar.utdallas.edu/event/ms_accounting_registrationcpa_advising_session_3439', 'snippet': 'MS Accounting Registration/CPA Advising Session. Wednesday, September 2, 2020 at 12:00pm to 2:00pm. Virtual Event. To help you review your CPA eligibility\xa0...'}, {'title': 'Staff List | Naveen Jindal School of Management - Dallas', 'link': 'https://jindal.utdallas.edu/staff-list/', 'snippet': 'Oct 20, 2021 ... Van Dam. Program Manager, Undergraduate Accounting Progr

(972) 883-2275

## How do I apply at UT Dallas

In [45]:
zero_shot_react_agent_response = zero_shot_react_agent(
    {
        "input": "How do I applying to UTD?"
    }
)



> Entering new  chain...
I don't know what UTD is.
Action: Dictionary for a word or phrases for general knowledge not related to UT Dallas
Action Input: UTD
Observation: {'UTD': None}
Thought:I still don't know what UTD is.
Action: UTD Search
Action Input: apply to UTD
Observation: {'items': [{'title': 'Apply', 'link': 'https://enroll.utdallas.edu/apply/', 'snippet': 'Submit an essay detailing the coursework you wish to complete at UT Dallas. Pay the $50 application fee. If you are interested in taking graduate classes or\xa0...'}, {'title': 'Steps To Apply - Graduate Admissions | The University of Texas at ...', 'link': 'https://graduate-admissions.utdallas.edu/apply-to-ut-dallas/apply/', 'snippet': 'Although UT Dallas sets minimum requirements, other application requirements vary by program. Check out the links below to learn what you will need to apply\xa0...'}, {'title': 'UT Dallas - Check Your Application Status', 'link': 'https://www.utdallas.edu/status/', 'snippet': 'Learn how

OutputParserException: Could not parse LLM output: `I found the following links:
https://enroll.utdallas.edu/apply/
https://graduate-admissions.utdallas.edu/apply-to-ut-dallas/apply/
https://www.utdallas.edu/status/
https://www.utdallas.edu/admissions/
https://enroll.utdallas.edu/freshman/application-process/
https://graduate-admissions.utdallas.edu/apply-to-ut-dallas/
https://graduate-admissions.utdallas.edu/`

## Who are the professors who research NLP at UTD

In [46]:
zero_shot_react_agent_response = zero_shot_react_agent(
    {
        "input": "Who are the professors who research NLP at UTD other than Vincent Ng"
    }
)



> Entering new  chain...
I need to find professors who research NLP at UTD
Action: UTD Search
Action Input: NLP
Observation: {'items': [{'title': 'Natural Language Processing (NLP) Archives - UT Dallas ...', 'link': 'https://cs.utdallas.edu/tag/natural-language-processing-nlp/', 'snippet': 'For almost 50 years, Dr. Klaus Truemper has worked at the University of Texas at Dallas: the first two decades in Operations…'}, {'title': 'Financial NLP - StARLinG', 'link': 'https://starling.utdallas.edu/datasets/fnlp/', 'snippet': 'Sep 25, 2019 ... Financial NLPPermalink. A relational dataset consisting of relations obtained by extracting information from SEC Form S-1 documents.'}, {'title': 'CS 6320.002 - Natural Language Processing (Spring 2023) at UT ...', 'link': 'https://personal.utdallas.edu/~sanda/courses/NLP/cs6320.html', 'snippet': 'Meeting times: M W 1:00 PM Classroom: ECSS 2.203. Instructor: Dr. Sanda Harabagiu( See personal Web page here ) Email: sanda@utdallas.edu'}, {'title': 'Nat

## Can you teel me the Difference Cs and SW at UTD

In [47]:
zero_shot_react_agent_response = zero_shot_react_agent(
    {
        "input": "Can you tell me the difference between Computer Science and  Software Engineering at UTD?"
    }
)



> Entering new  chain...
I don't know what Computer Science and  Software Engineering at UTD is.
Action: UTD Degree API
Action Input: Computer Science and  Software Engineering at UTD
Observation: {'items': [{'title': 'Software Engineering (BS) - UT Dallas 2022 Undergraduate Catalog', 'link': 'https://catalog.utdallas.edu/2022/undergraduate/programs/ecs/software-engineering', 'snippet': 'Jul 22, 2022 ... The Computer Science Department offers the BS degree in Computer Science and the BS degree in Software Engineering.'}, {'title': 'Software Engineering Program - UT Dallas 2021 Graduate Catalog ...', 'link': 'https://catalog.utdallas.edu/2021/graduate/programs/ecs/software-engineering', 'snippet': 'May 26, 2021 ... The Graduate Program in Computer Science provides intensive preparation in the design, programming, theory, and applications of computers. The\xa0...'}, {'title': 'Software Engineering Program - UT Dallas 2022 Graduate Catalog ...', 'link': 'https://catalog.utdallas.edu/202